## Future forecasting:

In [1]:
"""from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from xgboost import plot_importance
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso"""

In [1]:
## PULL Functions from custom functions
from custom_functions import bertopic_addons as cfc

## you will need to change the name below
currenttable = 'topified_vectorized_Science1900_2023'
aws_dfs = cfc.pull_aws_sql_database(currenttable)

Step 1/7: Loading the dataset ...


In [2]:
from custom_functions import time_series_addons as tsa
tsa.final_tableau_table(final_df= aws_dfs, final_forecasting = 5, final_moving_ave = 1)

-1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84


In [4]:
def cumsum_or_moving_average(infun_df, cumlative):
    # convert target topic to a moving average
    list_of_topic_codes = list(infun_df.columns.drop('date'))
    if cumlative == True:
        ## now predicting the moving average
        infun_df[list_of_topic_codes] = infun_df[list_of_topic_codes].cumsum()
    else:    
        ## now predicting the moving average
        infun_df[list_of_topic_codes] = infun_df[list_of_topic_codes].rolling(moving_average_years).mean()
    return infun_df

In [5]:
def clean_and_pivot_dataset(infun_df, infun_type_of_metric, infun_timeframe):
    
## you can change this to all topics 
    aws1_dfs= infun_df
    aws1_dfs['topic_code'] = pd.to_numeric(aws1_dfs['topic_code'], downcast='integer')
    aws1_dfs[infun_type_of_metric] = pd.to_numeric(aws1_dfs[infun_type_of_metric], downcast='integer')
    aws1_dfs = aws1_dfs[aws1_dfs['publicationDate'] != 'None'].reset_index()
    aws1_dfs['publicationDate'] = pd.to_datetime(aws1_dfs['publicationDate'], format='%Y-%m-%d')
    aws1_dfs['date'] = aws1_dfs['publicationDate'].dt.to_period(infun_timeframe)
    aws1_dfs['date']=aws1_dfs['date'].astype(str)
    aws1_dfs['date']=pd.to_datetime(aws1_dfs['date'])
    # ==== LEGACY NO NEED :)) aws1_dfs= aws1_dfs[aws1_dfs['topic_code']!=-1]

    ## code to change the date time, for the moment use year
    grouped_df = aws1_dfs.groupby(['topic_code', 'date'])[infun_type_of_metric].sum().reset_index()

    df = grouped_df.pivot(index='date', columns='topic_code', values=infun_type_of_metric)

    df.fillna(0, inplace=True)
    df = df.reset_index()
    return df

In [22]:
def plot_bertopic_data_into_future(dataframe, topic_number, future_years, moving_average, type_of_metric, timeframe, cumlative):
    
    ##### ============================ CLEANING UP DATAFRAME 
    # ## number of years into the future you can forecast
    if timeframe == 'M':
        n = future_years*12
        moving_average = moving_average*12
    else:
        n = future_years
        
    # Define the target column (Topic you want to forecast)
    target_topic = topic_number  # Change this to the Topic you want to forecast
    
    
    
    ## pivot df and clean the dataset
    pivot_df = clean_and_pivot_dataset(dataframe, type_of_metric, timeframe)
    # convert target topic to a moving average or cumalative 
    pivot_df = cumsum_or_moving_average(pivot_df, cumlative=cumlative)
    
    
    
    
    ## ================================  forecasting rearrangement

    # Dropping last n rows using drop
    target_column = pivot_df[target_topic]
    
    # forget about the lost values
    ## dropped the oldest columns
    target_column.drop(target_column.head(n).index, inplace = True)

    ## pivot_df['year'] + pd.offsets.DateOffset(years=5)
    untarget_columns = pivot_df.drop(target_topic, axis=1)

    ## keep this for later -- these are your forecasting columns 
    X_forecasting_data = untarget_columns.tail(n)
    X_forecasting_years = untarget_columns['date'].tail(n) + pd.offsets.DateOffset(years=future_years)
    X_forecasting_data['date'] = X_forecasting_years
    X_forecasting = X_forecasting_data.drop(columns=['date'])

    ##  drop the columns you keep for forecasting
    untarget_columns.drop(target_column.tail(n).index, inplace = True)

    df = pd.concat([untarget_columns, target_column.reset_index(drop=True)], axis=1)

    ## shift the prediction of each year
    df['date'] = df['date'] + pd.offsets.DateOffset(years=future_years)

    pivot_df = df    
    
    
    
    ### ================== TEST TRAIN SPLIT 
    # Separating the training set and testing set
    train_data=pivot_df[pivot_df['date'].dt.year<2011].reset_index(drop = True)
    test_data=pivot_df[pivot_df['date'].dt.year>2010].reset_index(drop = True)

    # Prepare the training and testing data
    X_train = train_data.drop(target_topic, axis=1)
    X_test = test_data.drop(target_topic, axis=1)

    # Shift the target column to align with next year's frequency
    y_train = train_data[target_topic].shift(-1).dropna()
    y_test = test_data[target_topic].shift(-1).dropna()

    # Exclude the 'YearMonth' column from the training and testing data
    X_train = train_data.drop(columns=['date', target_topic]).iloc[:-1]
    X_test = test_data.drop(columns=['date', target_topic]).iloc[:-1]
    
    
    
    
    ## ======================= MODEL 

    # Create an XGBoost regressor
    if cumlative == True: 
        model = Lasso()
    else:    
        model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=3)
    

    # Fit the model on the training data
    model.fit(X_train, y_train)

    # Make predictions on the testing data
    y_pred = model.predict(X_test)
    y_forecasting = model.predict(X_forecasting)

    # Create a DataFrame with YearMonth and the predicted values
    y_pred_df = pd.DataFrame({'date': test_data['date'].iloc[:-1], 'Predicted': y_pred})
    y_fore_df = pd.DataFrame({'date': X_forecasting_years, 'Forecasted': y_forecasting})

    # Merge the predicted DataFrame with the original test_data DataFrame
    merged_data = pd.merge(y_pred_df, test_data, on='date')
    merged_forecasted_data = pd.merge(y_fore_df, X_forecasting_data, on='date')

    #print(merged_forecasted_data)
    # Calculate the root mean squared error (RMSE)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    #print(f"Root Mean Squared Error (RMSE): {rmse}")
    
    
    
    ## =============================   BUILDING DATA TABLE 
    
    y_actual_test_df = test_data[['date',target_topic]]
    y_actual_train_df = train_data[['date',target_topic]]
    y_fore_df =y_fore_df.rename(columns={'Forecasted': target_topic})
    y_pred_df =y_pred_df.rename(columns={'Predicted': target_topic})

    y_actual_test_df['value'] = 'Actual_Test'
    y_actual_train_df['value'] = 'Actual_Train'
    y_fore_df['value'] = 'Forecasted'
    y_pred_df['value'] = 'Predicted_Test'

    final_data = pd.concat([y_actual_test_df,y_actual_train_df,y_fore_df,y_pred_df], axis=0)
    final_data =final_data.rename(columns={target_topic: 'quantity'})
    final_data['topic_code'] = target_topic
    final_data['RMSE'] = rmse
    final_data['metric'] = type_of_metric
    
    
    if cumlative == True: 
        final_data['cumsum_or_moving_average'] = 'cumsum'
    else: 
        final_data['cumsum_or_moving_average'] = 'moving_average'
    
    
    
    ### add moving average or cum sum
    
    
    
    """# Plot the predicted vs. actual values along with y_train
    plt.figure(figsize=(12, 6))
    plt.plot(merged_data['year'], merged_data['Predicted'], label='Predicted', marker='x')
    plt.plot(merged_data['year'], merged_data[target_topic], label='Actual', marker='o')
    plt.plot(train_data['year'].iloc[:-1], y_train, label='Train', linestyle='--', color='gray')
    plt.plot(merged_forecasted_data['year'], merged_forecasted_data['Forecasted'], label='Forecasted', marker='+')
    
    plt.xlabel('Time')
    plt.ylabel(type_of_metric)
    plt.title(f'Topic {target_topic} Citation Forecast (RMSE: {rmse:.2f})')
    plt.legend()
    plt.grid(True)
    # Feature importance plot (optional)
    plot_importance(model)
    plt.show()"""
    
    return final_data
    

In [7]:
"""## create an ordered list of topic codes 
topic_code_list = list(pd.Series(dataframe['topic_code'].sort_values()).unique())

## create a dataframe to concat later 
growing_df = pd.DataFrame()

## for loop through the topic code list 
for metric in ['count', 'citationCount', 'influentialCitationCount']:
    for topic_num in topic_code_list:
        current_df = plot_bertopic_data_into_future(dataframe, topic_num, 5, 4, metric, timeframe, cumlative=False)
        growing_df = pd.concat([current_df, growing_df]) 
    
## reset the index and drop the inxiex column 
growing_df = growing_df.reset_index().drop('index',axis=1)

## match the topic code to the topic description 
topic_codes = dataframe[['topic_code','topic_list']].drop_duplicates().reset_index().drop('index',axis=1)

## merge the topics descriptions to the data set
full_datase= growing_df.merge(topic_codes, on='topic_code')"""

"## create an ordered list of topic codes \ntopic_code_list = list(pd.Series(dataframe['topic_code'].sort_values()).unique())\n\n## create a dataframe to concat later \ngrowing_df = pd.DataFrame()\n\n## for loop through the topic code list \nfor metric in ['count', 'citationCount', 'influentialCitationCount']:\n    for topic_num in topic_code_list:\n        current_df = plot_bertopic_data_into_future(dataframe, topic_num, 5, 4, metric, timeframe, cumlative=False)\n        growing_df = pd.concat([current_df, growing_df]) \n    \n## reset the index and drop the inxiex column \ngrowing_df = growing_df.reset_index().drop('index',axis=1)\n\n## match the topic code to the topic description \ntopic_codes = dataframe[['topic_code','topic_list']].drop_duplicates().reset_index().drop('index',axis=1)\n\n## merge the topics descriptions to the data set\nfull_datase= growing_df.merge(topic_codes, on='topic_code')"

In [40]:
def tableau_tables(tab_df, tab_forecasting, tab_moving_ave, tab_cumsum):
    ## variables
    dataframe = tab_df
    forecasting_years = tab_forecasting
    moving_average_years = tab_moving_ave
    cumlative_boolean = tab_cumsum
    
    ## True goes with Monthly forecasting
    if cumlative_boolean == True: 
        timeframe = 'M'
    else: 
        timeframe = 'Y'
    
    ## 
    dataframe['count'] = 1

    ## create an ordered list of topic codes 
    topic_code_list = list(pd.Series(dataframe['topic_code'].sort_values()).unique())

    ## create a dataframe to concat later 
    growing_df = pd.DataFrame()

    ## for loop through the topic code list 
    for metric in ['count', 'citationCount', 'influentialCitationCount']:
        for topic_num in topic_code_list:
            print(topic_num)
            current_df = plot_bertopic_data_into_future(dataframe, topic_num, forecasting_years, moving_average_years, metric, timeframe, cumlative=cumlative_boolean)
            growing_df = pd.concat([current_df, growing_df]) 
        
    ## reset the index and drop the inxiex column 
    growing_df = growing_df.reset_index().drop('index',axis=1)

    ## match the topic code to the topic description 
    topic_codes = dataframe[['topic_code','topic_list']].drop_duplicates().reset_index().drop('index',axis=1)

    ## merge the topics descriptions to the data set
    full_dataset = growing_df.merge(topic_codes, on='topic_code')
    return full_dataset

In [41]:
def final_tableau_table(final_df, final_forecasting, final_moving_ave):
    cumsum_df = tableau_tables(tab_df= final_df, tab_forecasting= final_forecasting, tab_moving_ave= final_moving_ave, tab_cumsum= True)
    other_df = tableau_tables(tab_df= final_df, tab_forecasting= final_forecasting, tab_moving_ave= final_moving_ave, tab_cumsum= False)
    final_df = pd.concat([other_df, cumsum_df]) 
    return final_df
    

In [ ]:
dataframe = aws_dfs
forecasting_years = 5
moving_average_years = 1
## True goes with 
cumlative_boolean = True

In [42]:
final_tableau_table(final_df= aws_dfs, final_forecasting = 5, final_moving_ave = 1)

-1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
-1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
-1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
9

,date,quantity,value,topic_code,RMSE,metric,cumsum_or_moving_average,topic_list
0,2011-01-01,81.000000,Actual_Test,117,26.053664,influentialCitationCount,1,117_multiple sclerosis_sclerosis_multiple_axonal
1,2012-01-01,18.000000,Actual_Test,117,26.053664,influentialCitationCount,1,117_multiple sclerosis_sclerosis_multiple_axonal
2,2013-01-01,5.000000,Actual_Test,117,26.053664,influentialCitationCount,1,117_multiple sclerosis_sclerosis_multiple_axonal
3,2014-01-01,2.000000,Actual_Test,117,26.053664,influentialCitationCount,1,117_multiple sclerosis_sclerosis_multiple_axonal
4,2015-01-01,0.000000,Actual_Test,117,26.053664,influentialCitationCount,1,117_multiple sclerosis_sclerosis_multiple_axonal
...,...,...,...,...,...,...,...,...
583333,2023-04-01,19357.702272,Predicted_Test,-1,715.215259,count,cumsum,-1_patients_study_results_disease
583334,2023-05-01,19411.822625,Predicted_Test,-1,715.215259,count,cumsum,-1_patients_study_results_disease
583335,2023-06-01,19567.054142,Predicted_Test,-1,715.215259,count,cumsum,-1_patients_study_results_disease
583336,2023-07-01,19626.748011,Predicted_Test,-1,715.215259,count,cumsum,-1_patients_study_results_disease


In [13]:
from custom_functions import bertopic_addons as cfc

In [14]:
name_for_dataframe = 'forecasted_predicted_actual_Science_1900_2023'
cfc.dataframe_to_aws_sql(the_df,name_for_dataframe)
cfc.pull_aws_sql_database(name_for_dataframe)

NameError: name 'full_datase' is not defined